In [1]:
import pandas as pd
import lightgbm as lgb
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, LabelEncoder, KBinsDiscretizer
from sklearn.model_selection import KFold
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import mean_squared_error
import numpy as np
from pprint import pprint
from tqdm.auto import tqdm
from tqdm import tqdm_notebook
from scipy.sparse import hstack
from collections import defaultdict, Counter
from ds_tools.ds_tools import CategoricalTransformer
import matplotlib.pyplot as plt
import pickle
tqdm.pandas()

In [2]:
df_description = pd.read_csv('./data/data_definition.txt', sep='\t')

df_train_genba = pd.read_csv('./data/train_genba.tsv', sep='\t')
df_train_goto = pd.read_csv('./data/train_goto.tsv', sep='\t')

df_train = df_train_goto.merge(df_train_genba, on='pj_no', how='left')

df_test_genba = pd.read_csv('./data/test_genba.tsv', sep='\t')
df_test_goto = pd.read_csv('./data/test_goto.tsv', sep='\t')

df_test = df_test_goto.merge(df_test_genba, on='pj_no', how='left')
test_surface = df_test['tc_mseki']

df_train.drop(['id', 'kaoku_um', 'toshikuiki2', 'shu_sogi'], axis=1, inplace=True)
df_test.drop(['id', 'kaoku_um', 'toshikuiki2', 'shu_sogi'], axis=1, inplace=True)

## Preprocessing

In [3]:
def combine(row, combine_list, tup):
    l = set()
    for col in tup:
        if pd.notnull(row[col]):
            l.add(row[col])
    combine_list.append(','.join(l))

combine_cols = [('yoto', 100), ('road_hk', 100)]
for i, tup in enumerate([['yoto1', 'yoto2'], ['road1_hk', 'road2_hk', 'road3_hk', 'road4_hk']]):
    combine_train = []
    combine_test = []
    
    combine_col_name = combine_cols[i][0]
    _ = df_train.apply(lambda row: combine(row, combine_train, tup), axis=1)
    _ = df_test.apply(lambda row: combine(row, combine_test, tup), axis=1)

    count_vectorizer = CountVectorizer(min_df=combine_cols[i][1])
    combine_train_matrix = count_vectorizer.fit_transform(combine_train).todense()
    combine_test_matrix = count_vectorizer.transform(combine_test).todense()
    for i in range(combine_train_matrix.shape[1]):
        df_train['%s_%d' % (combine_col_name, i)] = combine_train_matrix[:, i]
        df_test['%s_%d' % (combine_col_name, i)] = combine_test_matrix[:, i]
    for col in tup:
        df_train.drop(col, axis=1, inplace=True)
        df_test.drop(col, axis=1, inplace=True)

In [4]:
for col in ['mseki_rd_hb', 'road3_fi', 'rosenka_hb', 'kempei2', 'road2_mg', 'kaoku_hb', 'bus_hon']:
    df_train[col].replace(0.0, np.nan, inplace=True)
    df_test[col].replace(0.0, np.nan, inplace=True)

coordinates = pickle.load(open('./data/coordinates.bin', 'rb'))

for df in [df_train, df_test]:
    df['station_name_prefix'] = df['rosen_nm1'].str.slice(stop=2)
    df.loc[pd.isnull(df['chiseki_kb_hb']) | (df['chiseki_kb_hb'] == 0), 'chiseki_kb_hb'] = df.loc[pd.isnull(df['chiseki_kb_hb']) | (df['chiseki_kb_hb'] == 0), 'chiseki_js_hb']
    df['chiseki_ratio'] = df['chiseki_js_hb']/df['chiseki_kb_hb']
    
    for col in ['magutchi']:
        col_mean = df_train[col].mean()
        df_train[col] = df_train[col].fillna(col_mean)
        df_test[col] = df_test[col].fillna(col_mean)
    
    df['lat'] = df['jukyo'].apply(lambda j: coordinates[j]['results'][0]['geometry']['location']['lat'] if coordinates[j]['results'] else np.nan)
    df['lng'] = df['jukyo'].apply(lambda j: coordinates[j]['results'][0]['geometry']['location']['lng'] if coordinates[j]['results'] else np.nan)
    
def fill_city_name(name):
    if '市' not in name and '郡' not in name:
        name = '市' + name
    return name

for df in [df_train, df_test]:
    df['jukyo'] = df['jukyo'].str.replace(r'[ヶｹ]', 'ケ')
    df['jukyo'] = df['jukyo'].apply(fill_city_name)
    city_split = df['jukyo'].str.split(r'[市郡]', n=1, expand=True)
    df['city'] = city_split[0]
    street_split = city_split[1].str.split(r'[町区]', n=1, expand=True)
    df['street'] = street_split[0]
    df['address_detail'] = street_split[1].str.strip().replace('', None)

In [5]:
splitter = KFold(n_splits=5, shuffle=True, random_state=28)
price_stats = []
for train_idx, valid_idx in splitter.split(df_train):
    price_stats_by_city = defaultdict(dict)
    for city, group in df_train.iloc[train_idx].groupby('city'):
        price_list = group['keiyaku_pr']/group['tc_mseki']
        price_stats_by_city[city]['price_by_city_mean'] = price_list.mean()
        price_stats_by_city[city]['price_by_city_median'] = price_list.median()
        price_stats_by_city[city]['price_by_city_min'] = price_list.min()
        price_stats_by_city[city]['price_by_city_max'] = price_list.max()
        price_stats_by_city[city]['price_by_city_std'] = price_list.std()
        price_stats_by_city[city]['price_by_city_count'] = len(price_list)
    for i, city in enumerate(df_train.iloc[valid_idx]['city']):
        price_stats.append((valid_idx[i], price_stats_by_city[city]))

price_stats_test = [] 
price_stats_by_city = defaultdict(dict)
for city, group in df_train.groupby('city'):
    price_list = group['keiyaku_pr']/group['tc_mseki']
    price_stats_by_city[city]['price_by_city_mean'] = price_list.mean()
    price_stats_by_city[city]['price_by_city_median'] = price_list.median()
    price_stats_by_city[city]['price_by_city_min'] = price_list.min()
    price_stats_by_city[city]['price_by_city_max'] = price_list.max()
    price_stats_by_city[city]['price_by_city_std'] = price_list.std()
    price_stats_by_city[city]['price_by_city_count'] = len(price_list)
for city in df_test['city']:
    price_stats_test.append(price_stats_by_city[city])
    
df_price_stats = pd.DataFrame([x[1] for x in sorted(price_stats, key=lambda x: x[0])])
df_price_stats_test = pd.DataFrame(price_stats_test)

df_train = pd.concat([df_train, df_price_stats], axis=1)
df_test = pd.concat([df_test, df_price_stats_test], axis=1)

## Train

In [6]:
continue_features = list(df_description[(df_description['データ型'] == '数値') & (df_description['項目名'] != 'pj_no')]['項目名'])
continue_features += ['price_by_city_mean', 'price_by_city_median', 'price_by_city_min', 'price_by_city_max', 
                      'price_by_city_std', 'price_by_city_count', 'chiseki_ratio', 'lng', 'lat']
objective = 'keiyaku_pr'
categorical_features = list(df_train)

for f in continue_features+[objective]:
    if f in categorical_features:
        categorical_features.remove(f)
        
for col in categorical_features:
    if col not in ['pj_no']:
        df_train[col] = df_train[col].fillna('UNKNOWN')
        df_test[col] = df_test[col].fillna('UNKNOWN')
        le = LabelEncoder()
        le.fit(pd.concat([df_train[col], df_test[col]]))
        df_train[col] = le.transform(df_train[col])
        df_test[col] = le.transform(df_test[col])
        
for col in continue_features:
    if col != 'keiyaku_pr':
        scaler = MinMaxScaler()
        df_train[col] = scaler.fit_transform(df_train[col].values.reshape(-1, 1))
        df_test[col] = scaler.transform(df_test[col].values.reshape(-1, 1))
    
df_test['keiyaku_pr'] = 0

## LGB

In [7]:
splitter = KFold(n_splits=5, shuffle=True, random_state=28)
prediction_list = []
best_scores = []
for train_idx, valid_idx in splitter.split(df_train):
    train, valid = df_train.iloc[train_idx], df_train.iloc[valid_idx]
    X_train, y_train = train.drop('keiyaku_pr', axis=1), np.log(train['keiyaku_pr']-1e6+1)
    X_valid, y_valid = valid.drop('keiyaku_pr', axis=1), np.log(valid['keiyaku_pr']-1e6+1)
    regressor = lgb.LGBMRegressor(n_estimators=100000, learning_rate=0.01, silent=False, random_state=28,
                                  bagging_fraction=0.8, bagging_freq=1, feature_fraction=0.8, lambda_l1=0.05, 
                                  max_depth=6, objective='fair')
    regressor.fit(X_train, y_train, eval_set=(X_valid, y_valid), early_stopping_rounds=500,
                  categorical_feature=categorical_features)
    prediction_list.append(regressor.predict(df_test[df_train.drop(objective, axis=1).columns]))
    best_scores.append(regressor.best_score_['valid_0']['fair'])

print("5-fold cv mean l2 %.8f" % np.mean(best_scores))

df_submission = pd.read_csv('./data/sample_submit.tsv', sep='\t', names=['id', 'pred'])

df_submission['pred'] = np.exp(np.mean(prediction_list, axis=0))-1+1e6
df_submission.to_csv('submission.tsv', sep='\t', header=None, index=False)

# 0.00499040

/usr/local/lib/python3.6/dist-packages/lightgbm/basic.py:1209: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['address_detail', 'bas_toho1', 'bas_toho2', 'bastei_nm1', 'bastei_nm2', 'bokachiiki', 'bus_yohi', 'chikukeikaku', 'city', 'eki_nm1', 'eki_nm2', 'fi3m_yohi', 'fi4m_yohi', 'gas', 'gesui', 'hiatari', 'hokakisei1', 'hokakisei2', 'hokakisei3', 'hokakisei4', 'hokakyoka', 'hw_status', 'jigata', 'josui', 'jukyo', 'kaihatsukyoka', 'kborjs', 'keikakuroad', 'kinshijiko', 'kobetsu1', 'kobetsu2', 'kobetsu3', 'kobetsu4', 'kodochiku', 'levelplan', 'pj_no', 'road1_sb', 'road2_sb', 'road3_sb', 'road4_sb', 'road_hk_0', 'road_hk_1', 'road_hk_2', 'road_hk_3', 'road_st', 'rosen_nm1', 'rosen_nm2', 'rs_e_kdate2', 'rs_e_kdate3', 'rs_e_m_ari', 'rs_e_m_nashi', 'rs_e_parking', 'rs_e_tahata', 'rs_e_zoki', 'rs_n_kdate2', 'rs_n_kdate3', 'rs_n_m_ari', 'rs_n_m_nashi', 'rs_n_parking', 'rs_n_tahata', 'rs_n_zoki', 'rs_s_kdate2', 'rs_s_kdate3', 'rs_s_m_ari', 'rs_s_m_nashi',

[1]	valid_0's fair: 0.0276382
Training until validation scores don't improve for 500 rounds.
[2]	valid_0's fair: 0.0272857
[3]	valid_0's fair: 0.0269638
[4]	valid_0's fair: 0.0266282
[5]	valid_0's fair: 0.0263382
[6]	valid_0's fair: 0.026043
[7]	valid_0's fair: 0.0257107
[8]	valid_0's fair: 0.0253957
[9]	valid_0's fair: 0.0250906
[10]	valid_0's fair: 0.024834
[11]	valid_0's fair: 0.0245677
[12]	valid_0's fair: 0.0242851
[13]	valid_0's fair: 0.0240024
[14]	valid_0's fair: 0.0237255
[15]	valid_0's fair: 0.0234547
[16]	valid_0's fair: 0.0231914
[17]	valid_0's fair: 0.022937
[18]	valid_0's fair: 0.0226743
[19]	valid_0's fair: 0.0224165
[20]	valid_0's fair: 0.0221731
[21]	valid_0's fair: 0.0219357
[22]	valid_0's fair: 0.0217043
[23]	valid_0's fair: 0.0214763
[24]	valid_0's fair: 0.0212533
[25]	valid_0's fair: 0.0210398
[26]	valid_0's fair: 0.0208236
[27]	valid_0's fair: 0.0206229
[28]	valid_0's fair: 0.0204191
[29]	valid_0's fair: 0.0202149
[30]	valid_0's fair: 0.0200111
[31]	valid_0's fair

[263]	valid_0's fair: 0.00828409
[264]	valid_0's fair: 0.0082753
[265]	valid_0's fair: 0.00826661
[266]	valid_0's fair: 0.00825623
[267]	valid_0's fair: 0.00824558
[268]	valid_0's fair: 0.00823855
[269]	valid_0's fair: 0.00822709
[270]	valid_0's fair: 0.00821694
[271]	valid_0's fair: 0.00820646
[272]	valid_0's fair: 0.00819756
[273]	valid_0's fair: 0.0081894
[274]	valid_0's fair: 0.00817905
[275]	valid_0's fair: 0.00817064
[276]	valid_0's fair: 0.00816117
[277]	valid_0's fair: 0.00815237
[278]	valid_0's fair: 0.00814554
[279]	valid_0's fair: 0.00813471
[280]	valid_0's fair: 0.00812474
[281]	valid_0's fair: 0.00811673
[282]	valid_0's fair: 0.00810797
[283]	valid_0's fair: 0.00810035
[284]	valid_0's fair: 0.00809202
[285]	valid_0's fair: 0.00808298
[286]	valid_0's fair: 0.00807498
[287]	valid_0's fair: 0.00806588
[288]	valid_0's fair: 0.00805937
[289]	valid_0's fair: 0.00805266
[290]	valid_0's fair: 0.00804303
[291]	valid_0's fair: 0.00803415
[292]	valid_0's fair: 0.00802653
[293]	valid_

[525]	valid_0's fair: 0.00697533
[526]	valid_0's fair: 0.00697179
[527]	valid_0's fair: 0.00696931
[528]	valid_0's fair: 0.00696746
[529]	valid_0's fair: 0.00696475
[530]	valid_0's fair: 0.0069622
[531]	valid_0's fair: 0.00695933
[532]	valid_0's fair: 0.00695658
[533]	valid_0's fair: 0.00695358
[534]	valid_0's fair: 0.00695191
[535]	valid_0's fair: 0.00694995
[536]	valid_0's fair: 0.00694736
[537]	valid_0's fair: 0.00694489
[538]	valid_0's fair: 0.00694289
[539]	valid_0's fair: 0.00694065
[540]	valid_0's fair: 0.00693867
[541]	valid_0's fair: 0.006936
[542]	valid_0's fair: 0.00693309
[543]	valid_0's fair: 0.00693101
[544]	valid_0's fair: 0.00692863
[545]	valid_0's fair: 0.00692555
[546]	valid_0's fair: 0.00692301
[547]	valid_0's fair: 0.00692117
[548]	valid_0's fair: 0.00692005
[549]	valid_0's fair: 0.00691653
[550]	valid_0's fair: 0.00691353
[551]	valid_0's fair: 0.00691173
[552]	valid_0's fair: 0.00690942
[553]	valid_0's fair: 0.00690822
[554]	valid_0's fair: 0.00690477
[555]	valid_0

[829]	valid_0's fair: 0.00650914
[830]	valid_0's fair: 0.00650825
[831]	valid_0's fair: 0.00650717
[832]	valid_0's fair: 0.00650621
[833]	valid_0's fair: 0.0065055
[834]	valid_0's fair: 0.00650485
[835]	valid_0's fair: 0.00650382
[836]	valid_0's fair: 0.00650293
[837]	valid_0's fair: 0.00650192
[838]	valid_0's fair: 0.0065016
[839]	valid_0's fair: 0.0064998
[840]	valid_0's fair: 0.0064989
[841]	valid_0's fair: 0.00649828
[842]	valid_0's fair: 0.0064974
[843]	valid_0's fair: 0.00649605
[844]	valid_0's fair: 0.00649552
[845]	valid_0's fair: 0.00649435
[846]	valid_0's fair: 0.00649365
[847]	valid_0's fair: 0.00649328
[848]	valid_0's fair: 0.0064923
[849]	valid_0's fair: 0.00649213
[850]	valid_0's fair: 0.00649238
[851]	valid_0's fair: 0.00649151
[852]	valid_0's fair: 0.00649016
[853]	valid_0's fair: 0.00648974
[854]	valid_0's fair: 0.00648879
[855]	valid_0's fair: 0.00648782
[856]	valid_0's fair: 0.00648718
[857]	valid_0's fair: 0.00648674
[858]	valid_0's fair: 0.00648539
[859]	valid_0's 

...

[2363]	valid_0's fair: 0.00520227
[2364]	valid_0's fair: 0.00520184
[2365]	valid_0's fair: 0.00520192
[2366]	valid_0's fair: 0.00520119
[2367]	valid_0's fair: 0.00520077
[2368]	valid_0's fair: 0.00520104
[2369]	valid_0's fair: 0.00520082
[2370]	valid_0's fair: 0.00520115
[2371]	valid_0's fair: 0.00520115
[2372]	valid_0's fair: 0.00520132
[2373]	valid_0's fair: 0.0052014
[2374]	valid_0's fair: 0.00520133
[2375]	valid_0's fair: 0.00520154
[2376]	valid_0's fair: 0.00520165
[2377]	valid_0's fair: 0.00520158
[2378]	valid_0's fair: 0.00520146
[2379]	valid_0's fair: 0.00520173
[2380]	valid_0's fair: 0.00520231
[2381]	valid_0's fair: 0.00520225
[2382]	valid_0's fair: 0.00520222
[2383]	valid_0's fair: 0.00520229
[2384]	valid_0's fair: 0.00520176
[2385]	valid_0's fair: 0.00520182
[2386]	valid_0's fair: 0.0052012
[2387]	valid_0's fair: 0.00520184
[2388]	valid_0's fair: 0.00520194
[2389]	valid_0's fair: 0.00520186
[2390]	valid_0's fair: 0.00520221
[2391]	valid_0's fair: 0.00520239
[2392]	valid_0's

[2665]	valid_0's fair: 0.00519716
[2666]	valid_0's fair: 0.00519664
[2667]	valid_0's fair: 0.00519639
[2668]	valid_0's fair: 0.00519594
[2669]	valid_0's fair: 0.00519588
[2670]	valid_0's fair: 0.0051961
[2671]	valid_0's fair: 0.00519594
[2672]	valid_0's fair: 0.00519599
[2673]	valid_0's fair: 0.00519579
[2674]	valid_0's fair: 0.00519615
[2675]	valid_0's fair: 0.00519566
[2676]	valid_0's fair: 0.00519578
[2677]	valid_0's fair: 0.00519585
[2678]	valid_0's fair: 0.00519594
[2679]	valid_0's fair: 0.00519594
[2680]	valid_0's fair: 0.0051955
[2681]	valid_0's fair: 0.00519558
[2682]	valid_0's fair: 0.00519582
[2683]	valid_0's fair: 0.00519586
[2684]	valid_0's fair: 0.00519562
[2685]	valid_0's fair: 0.00519567
[2686]	valid_0's fair: 0.00519537
[2687]	valid_0's fair: 0.00519537
[2688]	valid_0's fair: 0.00519499
[2689]	valid_0's fair: 0.00519509
[2690]	valid_0's fair: 0.00519461
[2691]	valid_0's fair: 0.00519476
[2692]	valid_0's fair: 0.00519492
[2693]	valid_0's fair: 0.00519491
[2694]	valid_0's

[2962]	valid_0's fair: 0.00518519
[2963]	valid_0's fair: 0.00518549
[2964]	valid_0's fair: 0.00518557
[2965]	valid_0's fair: 0.00518565
[2966]	valid_0's fair: 0.00518575
[2967]	valid_0's fair: 0.00518513
[2968]	valid_0's fair: 0.00518499
[2969]	valid_0's fair: 0.00518508
[2970]	valid_0's fair: 0.00518541
[2971]	valid_0's fair: 0.00518572
[2972]	valid_0's fair: 0.00518549
[2973]	valid_0's fair: 0.00518575
[2974]	valid_0's fair: 0.00518563
[2975]	valid_0's fair: 0.00518552
[2976]	valid_0's fair: 0.00518575
[2977]	valid_0's fair: 0.0051855
[2978]	valid_0's fair: 0.00518528
[2979]	valid_0's fair: 0.00518554
[2980]	valid_0's fair: 0.00518551
[2981]	valid_0's fair: 0.00518564
[2982]	valid_0's fair: 0.00518585
[2983]	valid_0's fair: 0.00518487
[2984]	valid_0's fair: 0.00518452
[2985]	valid_0's fair: 0.0051848
[2986]	valid_0's fair: 0.00518417
[2987]	valid_0's fair: 0.00518439
[2988]	valid_0's fair: 0.00518447
[2989]	valid_0's fair: 0.00518434
[2990]	valid_0's fair: 0.00518423
[2991]	valid_0's

[3228]	valid_0's fair: 0.00518135
[3229]	valid_0's fair: 0.00518151
[3230]	valid_0's fair: 0.00518163
[3231]	valid_0's fair: 0.00518172
[3232]	valid_0's fair: 0.00518156
[3233]	valid_0's fair: 0.00518165
[3234]	valid_0's fair: 0.00518143
[3235]	valid_0's fair: 0.00518185
[3236]	valid_0's fair: 0.00518165
[3237]	valid_0's fair: 0.0051818
[3238]	valid_0's fair: 0.00518182
[3239]	valid_0's fair: 0.00518184
[3240]	valid_0's fair: 0.00518175
[3241]	valid_0's fair: 0.00518179
[3242]	valid_0's fair: 0.00518156
[3243]	valid_0's fair: 0.00518166
[3244]	valid_0's fair: 0.0051817
[3245]	valid_0's fair: 0.00518173
[3246]	valid_0's fair: 0.0051816
[3247]	valid_0's fair: 0.00518192
[3248]	valid_0's fair: 0.00518154
[3249]	valid_0's fair: 0.00518118
[3250]	valid_0's fair: 0.00518146
[3251]	valid_0's fair: 0.00518148
[3252]	valid_0's fair: 0.00518149
[3253]	valid_0's fair: 0.00518123
[3254]	valid_0's fair: 0.0051812
[3255]	valid_0's fair: 0.00518164
[3256]	valid_0's fair: 0.00518165
[3257]	valid_0's f

[3532]	valid_0's fair: 0.00518349
[3533]	valid_0's fair: 0.00518333
[3534]	valid_0's fair: 0.00518343
[3535]	valid_0's fair: 0.00518356
[3536]	valid_0's fair: 0.00518345
[3537]	valid_0's fair: 0.0051836
[3538]	valid_0's fair: 0.00518345
[3539]	valid_0's fair: 0.00518328
[3540]	valid_0's fair: 0.0051829
[3541]	valid_0's fair: 0.00518305
[3542]	valid_0's fair: 0.00518302
[3543]	valid_0's fair: 0.00518303
[3544]	valid_0's fair: 0.00518298
[3545]	valid_0's fair: 0.00518326
[3546]	valid_0's fair: 0.00518305
[3547]	valid_0's fair: 0.00518284
[3548]	valid_0's fair: 0.00518284
[3549]	valid_0's fair: 0.00518313
[3550]	valid_0's fair: 0.00518307
[3551]	valid_0's fair: 0.00518337
[3552]	valid_0's fair: 0.00518344
[3553]	valid_0's fair: 0.00518344
[3554]	valid_0's fair: 0.00518334
[3555]	valid_0's fair: 0.00518348
[3556]	valid_0's fair: 0.00518334
[3557]	valid_0's fair: 0.00518356
[3558]	valid_0's fair: 0.00518367
[3559]	valid_0's fair: 0.00518368
[3560]	valid_0's fair: 0.00518354
[3561]	valid_0's

In [8]:
tmp = list(df_train)
tmp.remove('keiyaku_pr')

In [9]:
list(zip(tmp, regressor.feature_importances_))

[('pj_no', 1),
 ('tc_mseki', 3969),
 ('tt_mseki', 5340),
 ('levelplan', 533),
 ('fukuin', 1538),
 ('road_st', 3),
 ('magutchi', 5988),
 ('setsudo_hi', 2532),
 ('setsudo_kj', 601),
 ('jigata', 286),
 ('hiatari', 1005),
 ('niwasaki', 3695),
 ('garage', 497),
 ('kobetsu1', 1329),
 ('kobetsu2', 40),
 ('kobetsu3', 50),
 ('kobetsu4', 0),
 ('jukyo', 681),
 ('chiseki_js_hb', 1406),
 ('chiseki_kb_hb', 1151),
 ('kempei1', 60),
 ('kempei2', 23),
 ('yoseki1', 138),
 ('yoseki2', 154),
 ('josui', 0),
 ('gesui', 84),
 ('gas', 399),
 ('usui', 42),
 ('tateuri_su', 401),
 ('tochiuri_su', 201),
 ('joken_su', 6),
 ('hy1f_date_su', 0),
 ('hy2f_date_su', 231),
 ('hy3f_date_su', 37),
 ('road1_sb', 71),
 ('road1_fi', 1107),
 ('road1_mg', 1856),
 ('road2_sb', 149),
 ('road2_fi', 689),
 ('road2_mg', 1540),
 ('road3_sb', 22),
 ('road3_fi', 119),
 ('road3_mg', 227),
 ('road4_sb', 1),
 ('road4_fi', 30),
 ('road4_mg', 28),
 ('kaoku_hb', 1361),
 ('yheki_kotei', 812),
 ('yheki_umu', 32),
 ('yheki_yohi', 22),
 ('gk_sh